# Fine-tune BERT for Token Classification

In [1]:
### WARNING: Do NOT run this cell, unless you are running this on Google Colab. For a local installation run: pip install -r requirements.txt inside the terminal
!pip install -r requirements.txt
# % pip install transformers==4.9.1
# % pip install datasets==1.11.0
# % pip install tabulate==0.8.9
# % pip install seqeval==1.2.2

In [10]:
import random, time, os
import torch
from torch.nn import CrossEntropyLoss
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from transformers import BertForTokenClassification, AdamW
from transformers import get_linear_schedule_with_warmup
import logging, sys
from transformers import BertTokenizer

# Our code behind the scenes!
import bert_utils as utils

## Initialize Hyperparameters

In [ ]:
EPOCHS = 2
BERT_MODEL_NAME = "bert-base-cased"
GPU_RUN_IX=0

SEED_VAL = 1234500
SEQ_MAX_LEN = 256
PRINT_INFO_EVERY = 10 # Print status only every X batches
GRADIENT_CLIP = 1.0
LEARNING_RATE = 1e-5
BATCH_SIZE = 4

TRAIN_DATA_PATH = './data/conll2003.train.conll' #"data/trial_mini_data.conll" 

# IMPORTANT NOTE: We use as validation set the test portion, in order to avoid overfitting on the dev set, 
# and this way be able to evaluate later and compare with your previous models!
DEV_DATA_PATH = './data/conll2003.dev.conll' #"data/trial_mini_data.conll" # "

SAVE_MODEL_DIR = "saved_models/MY_BERT_NER/"

LABELS_FILENAME = f"{SAVE_MODEL_DIR}/label2index.json"
LOSS_TRN_FILENAME = f"{SAVE_MODEL_DIR}/Losses_Train_{EPOCHS}.json"
LOSS_DEV_FILENAME = f"{SAVE_MODEL_DIR}/Losses_Dev_{EPOCHS}.json"

PAD_TOKEN_LABEL_ID = CrossEntropyLoss().ignore_index # -100

if not os.path.exists(SAVE_MODEL_DIR):
    os.makedirs(SAVE_MODEL_DIR)


# Initialize Random seeds and validate if there's a GPU available...
device, USE_CUDA = utils.get_torch_device(GPU_RUN_IX)
random.seed(SEED_VAL)
np.random.seed(SEED_VAL)
torch.manual_seed(SEED_VAL)
torch.cuda.manual_seed_all(SEED_VAL)

## Record everything inside a Log File

In [ ]:
console_hdlr = logging.StreamHandler(sys.stdout)
file_hdlr = logging.FileHandler(filename=f"{SAVE_MODEL_DIR}/BERT_TokenClassifier_train_{EPOCHS}.log")
logging.basicConfig(level=logging.INFO, handlers=[console_hdlr, file_hdlr])
logging.info("Start Logging")

INFO:root:Start Logging


## Load Training and Validation Datasets

In [ ]:
# Initialize Tokenizer
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME, do_basic_tokenize=False)

# Load Train Dataset
train_data, train_labels, train_label2index = utils.read_conll(TRAIN_DATA_PATH, has_labels=True)
train_inputs, train_masks, train_labels, seq_lengths = utils.data_to_tensors(train_data, 
                                                                            tokenizer, 
                                                                            max_len=SEQ_MAX_LEN, 
                                                                            labels=train_labels, 
                                                                            label2index=train_label2index,
                                                                            pad_token_label_id=PAD_TOKEN_LABEL_ID)
utils.save_label_dict(train_label2index, filename=LABELS_FILENAME)
index2label = {v: k for k, v in train_label2index.items()}

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

# Load Dev Dataset
dev_data, dev_labels, _ = utils.read_conll(DEV_DATA_PATH, has_labels=True)
dev_inputs, dev_masks, dev_labels, dev_lens = utils.data_to_tensors(dev_data, 
                                                                    tokenizer, 
                                                                    max_len=SEQ_MAX_LEN, 
                                                                    labels=dev_labels, 
                                                                    label2index=train_label2index,
                                                                    pad_token_label_id=PAD_TOKEN_LABEL_ID)

# Create the DataLoader for our Development set.
dev_data = TensorDataset(dev_inputs, dev_masks, dev_labels, dev_lens)
dev_sampler = RandomSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=BATCH_SIZE)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:bert_utils:Read 14041 Sentences!
INFO:bert_utils:MAX TOKENIZED SEQ LENGTH IN DATASET IS 185
INFO:bert_utils:Read 3250 Sentences!
INFO:bert_utils:MAX TOKENIZED SEQ LENGTH IN DATASET IS 151


## Initialize Model Components

In [ ]:
model = BertForTokenClassification.from_pretrained(BERT_MODEL_NAME, num_labels=len(train_label2index))
model.config.finetuning_task = 'token-classification'
model.config.id2label = index2label
model.config.label2id = train_label2index
if USE_CUDA: model.cuda()

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * EPOCHS

# Create optimizer and the learning rate scheduler.
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                        num_warmup_steps=0,
                                        num_training_steps=total_steps)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

## Training Cycle (Fine-tunning)

In [ ]:
loss_trn_values, loss_dev_values = [], []


for epoch_i in range(1, EPOCHS+1):
    # Perform one full pass over the training set.
    logging.info("")
    logging.info('======== Epoch {:} / {:} ========'.format(epoch_i, EPOCHS))
    logging.info('Training...')

    t0 = time.time()
    total_loss = 0
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRADIENT_CLIP)

        # Update parameters
        optimizer.step()
        scheduler.step()

        # Progress update
        if step % PRINT_INFO_EVERY == 0 and step != 0:
            # Calculate elapsed time in minutes.
            elapsed = utils.format_time(time.time() - t0)
            # Report progress.
            logging.info('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.    Loss: {}.'.format(step, len(train_dataloader),
                                                                                            elapsed, loss.item()))

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)

    # Store the loss value for plotting the learning curve.
    loss_trn_values.append(avg_train_loss)

    logging.info("")
    logging.info("  Average training loss: {0:.4f}".format(avg_train_loss))
    logging.info("  Training Epoch took: {:}".format(utils.format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on our validation set.
    t0 = time.time()
    results, preds_list = utils.evaluate_bert_model(dev_dataloader, BATCH_SIZE, model, tokenizer, index2label, PAD_TOKEN_LABEL_ID, prefix="Validation Set")
    loss_dev_values.append(results['loss'])
    logging.info("  Validation Loss: {0:.2f}".format(results['loss']))
    logging.info("  Precision: {0:.2f} || Recall: {1:.2f} || F1: {2:.2f}".format(results['precision']*100, results['recall']*100, results['f1']*100))
    logging.info("  Validation took: {:}".format(utils.format_time(time.time() - t0)))


    # Save Checkpoint for this Epoch
    utils.save_model(f"{SAVE_MODEL_DIR}/EPOCH_{epoch_i}", {"args":[]}, model, tokenizer)


utils.save_losses(loss_trn_values, filename=LOSS_TRN_FILENAME)
utils.save_losses(loss_dev_values, filename=LOSS_DEV_FILENAME)
logging.info("")
logging.info("Training complete!")

INFO:root:
INFO:root:======== Epoch 1 / 2 ========
INFO:root:Training...
INFO:root:  Batch    10  of  3,511.    Elapsed: 0:00:52.    Loss: 1.7108650207519531.
INFO:root:  Batch    20  of  3,511.    Elapsed: 0:01:49.    Loss: 1.1141164302825928.
INFO:root:  Batch    30  of  3,511.    Elapsed: 0:02:46.    Loss: 0.7507561445236206.
INFO:root:  Batch    40  of  3,511.    Elapsed: 0:03:26.    Loss: 0.5974534749984741.
INFO:root:  Batch    50  of  3,511.    Elapsed: 0:04:09.    Loss: 0.6428341865539551.
INFO:root:  Batch    60  of  3,511.    Elapsed: 0:04:53.    Loss: 0.5562766790390015.
INFO:root:  Batch    70  of  3,511.    Elapsed: 0:05:34.    Loss: 0.5168237686157227.
INFO:root:  Batch    80  of  3,511.    Elapsed: 0:06:15.    Loss: 0.17914146184921265.
INFO:root:  Batch    90  of  3,511.    Elapsed: 0:06:55.    Loss: 0.5947015881538391.
INFO:root:  Batch   100  of  3,511.    Elapsed: 0:07:37.    Loss: 0.3190470039844513.
INFO:root:  Batch   110  of  3,511.    Elapsed: 0:08:20.    Loss: 

# Use Fine-tuned Model for Predictions

In [ ]:
from transformers import pipeline
from torch.utils.data import SequentialSampler

GPU_IX=0
device, USE_CUDA = utils.get_torch_device(GPU_IX)
FILE_HAS_GOLD = True
SEQ_MAX_LEN = 256
BATCH_SIZE = 4
# IMPORTANT NOTE: We predict on the dev set to make the results comparable with your previous models from this course
TEST_DATA_PATH = "./data/conll2003.dev.conll" #"data/trial_mini_data.conll" # 
# TEST_DATA_PATH = "data/trial_unk_data.conll"
MODEL_DIR = "saved_models/MY_BERT_NER/"
LOAD_EPOCH = 1
INPUTS_PATH=f"{MODEL_DIR}/EPOCH_{LOAD_EPOCH}/model_inputs.txt"
OUTPUTS_PATH=f"{MODEL_DIR}/EPOCH_{LOAD_EPOCH}/model_outputs.txt"
PAD_TOKEN_LABEL_ID = CrossEntropyLoss().ignore_index # -100

console_hdlr = logging.StreamHandler(sys.stdout)
file_hdlr = logging.FileHandler(filename=f"{MODEL_DIR}/EPOCH_{LOAD_EPOCH}/BERT_TokenClassifier_predictions.log")
logging.basicConfig(level=logging.INFO, handlers=[console_hdlr, file_hdlr])

## Load Pre-trained Model

In [ ]:
model, tokenizer = utils.load_model(BertForTokenClassification, BertTokenizer, f"{MODEL_DIR}/EPOCH_{LOAD_EPOCH}")
label2index = utils.load_label_dict(f"{MODEL_DIR}/label2index.json")
index2label = {v:k for k,v in label2index.items()}

## Load File for Predictions 

In [ ]:
test_data, test_labels, _ = utils.read_conll(TEST_DATA_PATH, has_labels=FILE_HAS_GOLD)
prediction_inputs, prediction_masks, gold_labels, seq_lens = utils.data_to_tensors(test_data, 
                                                                                   tokenizer, 
                                                                                   max_len=SEQ_MAX_LEN, 
                                                                                   labels=test_labels, 
                                                                                   label2index=label2index)

INFO:bert_utils:Read 3250 Sentences!
INFO:bert_utils:MAX TOKENIZED SEQ LENGTH IN DATASET IS 151


## Make Predictions

In [ ]:
if FILE_HAS_GOLD:
    prediction_data = TensorDataset(prediction_inputs, prediction_masks, gold_labels, seq_lens)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=BATCH_SIZE)

    logging.info('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
    
    results, preds_list = utils.evaluate_bert_model(prediction_dataloader, BATCH_SIZE, model, tokenizer, index2label, 
                                                        PAD_TOKEN_LABEL_ID, full_report=True, prefix="Test Set")
    logging.info("  Test Loss: {0:.2f}".format(results['loss']))
    logging.info("  Precision: {0:.2f} || Recall: {1:.2f} || F1: {2:.2f}".format(results['precision']*100, results['recall']*100, results['f1']*100))

    with open(OUTPUTS_PATH, "w") as fout:
        with open(INPUTS_PATH, "w") as fin:
            for sent, pred in preds_list:
                fin.write(" ".join(sent)+"\n")
                fout.write(" ".join(pred)+"\n")

else:
    # https://huggingface.co/transformers/main_classes/pipelines.html#transformers.TokenClassificationPipeline
    logging.info('Predicting labels for {:,} test sentences...'.format(len(test_data)))
    if not USE_CUDA: GPU_IX = -1
    nlp = pipeline('token-classification', model=model, tokenizer=tokenizer, device=GPU_IX)
    nlp.ignore_labels = []
    with open(OUTPUTS_PATH, "w") as fout:
        with open(INPUTS_PATH, "w") as fin:
            for seq_ix, seq in enumerate(test_data):
                sentence = " ".join(seq)
                predicted_labels = []
                output_obj = nlp(sentence)
                # [print(o) for o in output_obj]
                for tok in output_obj:
                    if '##' not in tok['word']:
                        predicted_labels.append(tok['entity'])
                logging.info(f"\n----- {seq_ix+1} -----\n{seq}\nPRED:{predicted_labels}")
                fin.write(sentence+"\n")
                fout.write(" ".join(predicted_labels)+"\n")

INFO:root:Predicting labels for 3,250 test sentences...
INFO:bert_utils:***** Running evaluation Test Set *****
INFO:bert_utils:  Batch size = 4
INFO:bert_utils:{0: 'B-ORG', 1: 'O', 2: 'B-MISC', 3: 'B-PER', 4: 'I-PER', 5: 'B-LOC', 6: 'I-ORG', 7: 'I-MISC', 8: 'I-LOC'}
INFO:bert_utils:
----- 1 -----
['CRICKET', '-', 'LEICESTERSHIRE', 'TAKE', 'OVER', 'AT', 'TOP', 'AFTER', 'INNINGS', 'VICTORY', '.']

GOLD: ['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
PRED:['O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

INFO:bert_utils:
----- 2 -----
['LONDON', '1996-08-30']

GOLD: ['B-LOC', 'O']
PRED:['B-LOC', 'O']

INFO:bert_utils:
----- 3 -----
['West', 'Indian', 'all-rounder', 'Phil', 'Simmons', 'took', 'four', 'for', '38', 'on', 'Friday', 'as', 'Leicestershire', 'beat', 'Somerset', 'by', 'an', 'innings', 'and', '39', 'runs', 'in', 'two', 'days', 'to', 'take', 'over', 'at', 'the', 'head', 'of', 'the', 'county', 'championship', '.']

GOLD: ['B-MISC', 'I-MISC', 'O', 'B-PER', 'I